# Checkpoint 1

**Due: Tuesday, 17 October, 2023 at 11:00am BST**

Total points: 100

### Read This First
1. The only external Python packages you may use are `matplotlib`, `numpy`, and `scipy`.

1. Use the constants provided in the cells. Do not use your own constants.

1. Wherever you see `raise NotImplementedError()`, remove that line and put your code there.

1. Put the code that produces the output for a given task in the cell indicated. 

1. You are welcome to add as many cells as you like for imports, new function definitions, variables, etc.

1. Your notebook must run correctly when executed once from start to finish. Your notebook will be graded based on how it runs, not how it looks when you submit it. To test this, go to the *Kernel* menu and select *Restart & Run All*.

1. Once you are happy with it, clear the output by selecting *Restart & Clear Output* from the *Kernel* menu.

1. Submit through Noteable.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import time
import scipy
from scipy import sparse
from scipy.sparse import linalg

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 14

# Problem 1 - 20 points

You need to write a function that takes as input two filenames and 
* reads the x and y values from the first file 
* reads the array xnew values from the second file
* returns the result of interpolation of x,y values on the xnew array (as a numpy array)
* plots the x,y values as a scatterplot, and overplots the interpolated curve

Your code *must* chose the best interpolation method for a given dataset. 
Use *only* interpolation methods discussed in the workshop.

You are provided with two ascii files that you can test your code with:
- checkpoint1_problem1_file1.txt
- checkpoint1_problem1_file2.txt

The evaluation will be done using similar files that we do not share with you. 

You should expect that the y-values in the data should be in the range  of approximately $-150..150$

We expect this code should *not* run more than 60 seconds

The result will be be verified to provide a mean square error (MSE) with respect to the true function of MSE < 100 , where

$
\large
\begin{align}
MSE = \frac{1}{N} \sum_{i=1}^{N} (y_{interp, i} - y_{true, i})^2.
\end{align}
$

In [ ]:
def solve_interpolator(file1, file2):
    """
    This function *must* return numpy array of interpolated values
    DO NOT MODIFY the name and arguments of the function
    """
    # load data
    x, y = np.loadtxt(file1) 
    xnew = np.loadtxt(file2)
    ind = np.argsort(x)
    xind = x[ind]
    yind = y[ind]
    xnewind = np.argsort(xnew)
    xnew = xnew[xnewind]
     
    minx = int(np.ceil(min(x)))
    maxx = int(np.floor(max(x)))
    
    uvs = scipy.interpolate.UnivariateSpline(xind,yind, s=100000000)
    testinglistx = []
    traininglistx = []
    testinglisty = []
    traininglisty = []
    inds = np.arange(0,np.size(x))
    
    for i in range(10):
        testingx = (x[10*(i-1):10*i])
        testinds = np.argsort(testingx)
        testingx = testingx[testinds]
        testingy = (y[10*(i-1):10*i])
        testingy = testingy[testinds]
        testinglistx.append(testingx)
        testinglisty.append(testingy)
        trainingx = np.delete(x, inds[10*(i-1):10*i])
        trainingy = np.delete(x, inds[10*(i-1):10*i])
        traininds = np.argsort(trainingx)
        trainingx = trainingx[traininds]   
        trainingy = trainingy[traininds]
        traininglistx.append(trainingx)
        traininglisty.append(trainingy)
    
    maxx = int(np.floor(max(x)))
    minx = int(np.ceil(min(x)))
    curve_dataset_x = np.linspace(minx, maxx, maxx-minx )
    curve_dataset_y = uvs(curve_dataset_x)
     
    uvs_mse_list = []
    lin_mse_list = []
    cs_mse_list = []
    
    for j in range(10):
        
        # univariate spline interpolation
        uvs = scipy.interpolate.UnivariateSpline(traininglistx[i],traininglisty[i], s=100000000)
        ynewuvs = uvs(curve_dataset_x)
        
        # linear interpolation
        lin = scipy.interpolate.interp1d(traininglistx[i], traininglisty[i])
        ynewlin = lin(curve_dataset_x)

        # cubic spline interpolation
        cs = scipy.interpolate.CubicSpline(traininglistx[i], traininglisty[i])
        ynewcs = cs(curve_dataset_x)
        
        uvs_mse = mean_square_error(testinglistx[i], testinglisty[i], uvs)
        uvs_mse_list.append(uvs_mse)
        lin_mse = mean_square_error(testinglistx[i], testinglisty[i], lin)
        lin_mse_list.append(lin_mse)
        cs_mse = mean_square_error(testinglistx[i], testinglisty[i], cs)
        cs_mse_list.append(cs_mse)
    
    uvs_av = sum(uvs_mse_list)/len(uvs_mse_list)
    lin_av = sum(lin_mse_list)/len(lin_mse_list)
    cs_av = sum(cs_mse_list)/len(cs_mse_list)
    
    av_list = [uvs_av, lin_av, cs_av]
    bestmodel = min(av_list)
    
    # plot data and interpolated curves
    plt.plot(curve_dataset_x, curve_dataset_y, '-', label = 'Univariate spline')
    plt.plot(x, y, 'o', label = 'data')
    plt.title('Interpolation Methods on Data Set')
    plt.xlabel('x values')
    plt.ylabel('y values')
    plt.legend()
    plt.show()
    
def mean_square_error(listx, listy, model):
    
    sum = 0
    for i in range(len(listx)):
        predictedy = model(listx[i])
        sum += (predictedy - listy[i]) ** 2

    mse = sum / len(listx)
    
    return mse
     

The cell below will run tests when grading.

In [ ]:
print ("Testing, testing...")
# This should run and return an array and do the plot
pred = solve_interpolator('checkpoint1_problem1_file1.txt', 'checkpoint1_problem1_file2.txt')

# Below we run additional hidden tests

# Problem 2 - 65 points

This problem is divided into 5 tasks, worth the following point values:

1. 15 points
2. 10 points
3. 10 points
4. 15 points
5. 15 points

## The 1D time-independent Schrödinger equation

In one dimension, the time-independent Schrödinger equation is given by

$
\large
\begin{align}
\mathbf{H}\ \mathbf{\Psi} = E\ \mathbf{\Psi}
\end{align}
$,

where $\mathbf{H}$ is the Hamiltonian. Here, $E$ and $\mathbf{\Psi}$ are the eigenvectors and eigenvalues of $\mathbf{H}$, respectively. The lowest values of $E$ correspond to the lowest energy levels of the hydrogen atom.

The theoretical values for the energy levels are given by

$
\Large
\begin{align}
e_{n} = -\frac{c_2}{2 r_0 n^2},
\end{align}
$

where $n$ is the energy level (i.e., 1, 2, 3,...), $c_2$ is a constant defined below, and $r_{0}$ is the Bohr radius, given by

$
\Large
\begin{align}
r_{0} = \frac{4 \pi \epsilon_{0} \hbar^{2}}{m e^{2}}.
\end{align}
$


The Hamiltonian is expressed as

$
\Large
\begin{align}
H = -\frac{\hbar^2}{2m} \nabla^2 + V(r),
\end{align}
$

where $V(r)$ is the electric potential energy, given by

$
\Large
\begin{align}
V(r) = -\frac{e^{2}}{4 \pi \epsilon_{0} r}.
\end{align}
$

In matrix form, the Schrödinger equation is solved for N equally spaced values of r, such that r goes from ($r_{max}$/N) to $r_{max}$, where $r_{max} \sim 1.5$ nm is a sensible choice. To turn the Schrödinger equation into a matrix, $\textbf{V(r)}$ should be a diagonal matrix with the values of the potential at each r along the diagonal.

For this problem, the constants for the above equations have been defined for you in the cell below. Please use these for your calculations.
* $\frac{\hbar^{2}}{2m} = 0.0380998\ nm^{2} eV$ (called `c1` below)
* $\frac{e^{2}}{4 \pi \epsilon_{0}} = 1.43996\ nm\ eV$ (called `c2` below)
* $r_{0} = 0.0529177\ nm$ (the Bohr radius, called `r0` below)
* Planck constant $h = 6.62606896\times10^{-34} J s$ (`h`)
* Speed of light $c = 299792458\ m/s$ (`c`)

In [ ]:
# Constants (use these)
c1 = 0.0380998 # nm^2 eV
c2 = 1.43996 # nm eV
r0 = 0.0529177 # nm
h  = 6.62606896e-34 # J s
c  = 299792458. # m/s
hc = 1239.8419 # eV nm

# Task 1 - 15 points

Write a function to calculate and return the two lowest energy values of the hydrogen atom. Your function should take one argument corresponding to the desired tolerance. The tolerance for a given energy level, n, should be calculated as

$
\begin{align}
tol_n = \big{|}\frac{E_{n,calc} - E_{n,theo}}{E_{n,theo}}\big{|},
\end{align}
$

where $E_{n,calc}$ and $E_{n,theo}$ are the calculated and theoretical energies for level n. Both energy levels must be within the desired tolerance.

Your function should automatically determine the minimum size of the matrix, $N$, that yields the desired results. The value of $N$ must be within a factor of 2 or the correct value, which is not given. Start at a reasonably low value, $N \sim 100$, and increase from there. For efficiency, try to implement your function to take as few steps as possible to reach the desired tolerance.

Implement your function in the cell below. Do not alter the arguments of the function. The cell after that gives an example of how the function will be called. The code will be run with a different tolerance value and will be marked based on whether it achieves the desired tolerance and if the value of N is within a factor of two of the lowest value.

Your function should complete in less than 60 seconds.

In [ ]:
def calculate_energy_levels(tol):
        
    N = 100
    
    while N < 10000:
        
        rmax = 1.5
        r = np.linspace(rmax/N, rmax, N)

        V_values = -c2/r
        V = scipy.sparse.diags(V_values, offsets = 0, shape=(N,N))

        LP_diags = [1,-2,1]
        LP_offsets = [-1,0,1]
        laplacian = scipy.sparse.diags(LP_diags, LP_offsets, shape=(N,N))

        H = -c1*(1/(rmax/N)**2)*laplacian + V
        evals = linalg.eigsh(H, k=2, which = 'SA', return_eigenvectors = False)
        E = np.sort(np.real(evals))
        E1 = E[0]
        E2 = E[1] 
        
        Eth1 = -c2 / (2 * r0)
        er1 = abs((E1 - Eth1) / Eth1)
        
        Eth2 = Eth1 / 4
        er2 = abs((E2 - Eth2) / Eth2)
        if er1 <= tol and er2 <= tol:
            break
        
        N *= 2
    print(f"Minimum size of matrix: {N = }")   
    return E1, E2

The cell below should run your code correctly. Additional tests with other tolerance values will be used for assessment.

In [ ]:
def check_p2t1(tol):
    t1 = time.time()
    E1, E2 = calculate_energy_levels(tol)
    t2 = time.time()
    print(f"Calculation time: {t2 - t1} seconds.")

    Eth1 = -c2 / (2 * r0)
    er1 = abs((E1 - Eth1) / Eth1)
    print(f"{E1 = }, error = {er1}")
    assert er1 <= tol

    Eth2 = Eth1 / 4
    er2 = abs((E2 - Eth2) / Eth2)
    print(f"{E2 = }, error = {er2}")
    assert er2 <= tol

my_tol = 1e-3
check_p2t1(my_tol)

## Task 2 - 10 points

Now, imagine the Coulomb law has a minor modification to it, and is now given by:

$
\Large
\begin{align}
F(r) = -\frac{e^{2}}{4 \pi \epsilon_{0} r^{2}} \left( \frac{r}{r_{0}} \right)^{\alpha},
\end{align}
$

where $\alpha$ is some constant and $r_{0}$ is the Bohr radius, given above.

The electric potential is given by:

$
\Large
\begin{align}
V(r) = \int_{r}^{\infty} F(r^{\prime}) dr^{\prime}
\end{align}
$

Using the constants defined previously, write a function to calculate V(r) using the modified Coulomb law by numerically solving the equation above. This function need only accept a single value of radius and not an entire array. Your function must agree with the analytical value to within $10^{-5}$ eV.

Your function should go in the cell below using the template for `potential_numerical`. Do not modify the list of arguments for the function.

In [ ]:
def f(r, alpha):
    return r**(alpha-2)

def potential_numerical(r, alpha):
    constant = -c2/(r0**alpha) # constant outside integral
    sol,err = scipy.integrate.quad(f, r, np.inf, args=(alpha))
    I = sol*constant
    return I
    

The cell below will test your function for $\alpha=0.01$ for a few values of radius.

In [ ]:
def potential_exact(r, alpha):
    return c2*np.power(r,alpha-1)*np.power(r0,-alpha) / (alpha-1)

my_alpha = 0.01
for my_r in np.linspace(0.01, 1, 100):
    diff = abs(potential_numerical(my_r, my_alpha) - potential_exact(my_r, my_alpha))
    assert(diff <= 1e-5)

## Task 3 - 10 points

Make a plot showing the difference between the modified potential with $\alpha=0.01$ and the classical (unmodified) potential (i.e., $|V_{mod} - V|$) over the range of r values used in Task 1. Make the plot in log-space so the difference can be seen clearly. Plot separate curves for $V_{mod} > V$ and $V_{mod} < V$ and add a legend to show which is which. Include appropriate axes labels and units.

In [ ]:
from scipy import sparse

alpha = 0.01
N = 800
rmax = 1.5
r = np.linspace(rmax/N, rmax, N)
V = -c2/r
Vmod = potential_exact(r,alpha)
absdiff = abs(Vmod - V)

plt.loglog(r[0:10], absdiff[0:10], label = 'Vmod > V' )
plt.loglog(r[9:800], absdiff[9:800], label = 'Vmod < V')
plt.loglog(r, Vmod)
plt.title('Absolute difference of modified vs unmodified potential [alpha = 0.01]')
plt.xlabel('r [nm]')
plt.ylabel('|V_mod - V|')
plt.legend()
plt.show()

## Task 4 - 15 points

Write a function to calculate the first 2 energy levels (eigenvalues of $H$) for some value of $\alpha$ and print out the values in eV. Use a matrix size of $N = 1024$. The values must be accurate to 0.05% (i.e., $5\times 10^{-4}$, following the definition in Task 1). Use the function template `calculate_energy_levels_modified` below for your function. It is fine to call functions you've already written. Do not alter the argument list. Your function will be tested with other values of $\alpha$.

In the cell after, plot the difference $\Delta E$ between the two lowest energy levels as a function of $\alpha$ for 10 values of $\alpha$ in the range [0, 0.01]. Remember axes labels and units.

In [ ]:
def calculate_energy_levels_modified(alpha):
    
    N = 1024
    rmax = 1.5
    r = np.linspace(rmax/N, rmax, N)
    LP_diags = [1,-2,1]
    LP_offsets = [-1,0,1]
    laplacian = scipy.sparse.diags(LP_diags, LP_offsets, shape=(N,N))
    
    V_values = potential_exact(r, alpha)
    V = scipy.sparse.diags(V_values, offsets = 0, shape=(N,N))
    H = -c1*(1/(rmax/N)**2)*laplacian + V
    evals = linalg.eigsh(H, k=2, which = 'SA', return_eigenvectors = False)
    E = np.sort(np.real(evals))
    E1 = E[0]
    E2 = E[1] 
    
    
    return  E1, E2

The cell below will test your function against various values of $\alpha$.

In [ ]:
alpha = 0.01

In [ ]:
my_alpha = 0.01
E1, E2 = calculate_energy_levels_modified(my_alpha)
print (f"alpha = {alpha}: {E1 = }, {E2 = }")

In the cell below, make the plot of $\Delta E$ vs. $\alpha$ as instructed above.

In [ ]:
alpha = np.linspace(0, 0.01, 10)
deltaElist = []

for i in range(10):
    E1, E2 = calculate_energy_levels_modified(alpha[i])
    deltaE = E2 - E1
    deltaElist.append(deltaE)
    

In [ ]:
plt.plot(deltaElist, alpha)
plt.title('$\Delta$E as a function of alpha')
plt.xlabel('$\Delta$E [nm eV]')
plt.ylabel('alpha [dimensionless]')
plt.show()

## Task 5 - 15 points

The transition between the 1st and 2nd states is known as the Lyman-$\alpha$ transition. The photon emitted by this transition will have a wavelength, $\lambda$, given by

$
\Large
\begin{align}
\lambda = \frac{hc}{\Delta E}.
\end{align}
$

Imagine the wavelength of this transition has been measured to some value, $\lambda$. Using the template below, write a function to compute the value of $\alpha$ for a given value of $\lambda$. Assume that $\alpha$ is in the range [0, 0.1]. Your function will be tested with a random value of $\lambda$. Your function must complete in 60 seconds or less and return a value that is accurate to within 1% of the correct answer. Do not alter the list of arguments in the function, `calculate_alpha`.

In [ ]:
# Constants (use these)
c1 = 0.0380998 # nm^2 eV
c2 = 1.43996 # nm eV
r0 = 0.0529177 # nm
h  = 6.62606896e-34 # J s
c  = 299792458. # m/s
hc = 1239.8419 # eV nm

In [ ]:
def calculate_alpha(my_lambda):
    
    my_deltaE = h*c/my_lambda
    tol = 0.001
    alpha = 0.05
    
    for i in range(100):
        
        E1, E2 = calculate_energy_levels_modified(alpha)
        deltaEcalc = E2 - E1
        diff = abs(my_deltaE - deltaEcalc)
        if diff <= tol:
            break
        if deltaEcalc < my_deltaE:
            alpha = alpha + alpha/2
        else:
            alpha = alpha - alpha/2
    
    return alpha

The cell below will run your function against additional values.

In [ ]:
my_lambda = 118.22870362
my_alpha = calculate_alpha(my_lambda)
print (f"{my_lambda = }, {my_alpha = }.")

# Problem 3 - 15 points

The equation for an ellipsoid is given by:

$
\begin{align}
\large
\frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} = 1,
\end{align}
$

where $a$, $b$, and $c$ are the lengths of the semi-axes with $a > b > c$. Here, $x$, $y$, and $z$ are arbitrarily oriented 3D Cartesian coordinates.

Included with the checkpoint is a file, called **ellipse.txt**, containing coordinates of points associated with a 3D multivariate normal distribution. The lengths of the semi-axes correspond to the standard deviations of the distributions transformed into the principal axes. Using the template below for `calculate_ellipse_size`, write a function to read in the data from the file and output $a$, $b$, and $c$ such that $a > b > c$. Your code will be tested with a different file of the same format. The values must be within 1% of the correct answers. Do not alter the argument list of the function.

In [ ]:
def calculate_ellipse_size(filename):
    
    x = np.loadtxt(filename, usecols = 0)
    y = np.loadtxt(filename, usecols = 1)
    z = np.loadtxt(filename, usecols = 2)

    p = np.array([x,y,z])
    cov = np.cov(p)
    
    l, v = scipy.linalg.eig(cov)
    pnew = v.T @ p
    
    evals = pnew.std(axis=1)
    evals = np.sort(evals)
    evals = evals[::-1]
    a = evals[0]
    b = evals[1]
    c = evals[2]
    
    return a, b, c

In [ ]:
a, b, c = calculate_ellipse_size("ellipse.txt")
print (f"{a = }, {b = }, {c = }.")